# BERT

참고한 BERT 사용법 : https://velog.io/@seolini43/일상연애-주제의-한국어-대화-BERT로-이진-분류-모델-만들기파이썬Colab-코드

## (1) 학습데이터 불러오기

In [ ]:
#필요한 라이브러리 불러오기
import kss
import os
import csv
import pandas as pd
import re
import tensorflow as tf
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import random
import time
import datetime

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 636 kB 50.5 MB/s 
     |████████████████████████████████| 3.3 MB 22.2 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
device #실제 프로젝트에서는 GPU 활용했음

device(type='cpu')

In [ ]:
#치안데이터 불러오기
chian_data = pd.read_csv('chian2021-08-23-15-19-08-경남신문/2021-08-23-15-19-08-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(chian_data)):
  chian_data['뉴스내용'][i] = re.sub("[\[\]]","",chian_data['뉴스내용'][i])

#문장단위 토크나이징
chian_sentences  = []
for i in range(0,len(chian_data)):
  chian_sentences.append(kss.split_sentences(chian_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
chian_sentences = sum(chian_sentences , [])

#치안데이터 라벨링(True)
chian = pd.DataFrame(chian_sentences)
chian['label'] = 0
chian.loc[(chian['label'] == 0), 'label'] = 1
chian = chian.rename(columns={0:'문장'})
chian.to_csv('치안뉴스라벨링.csv')

In [ ]:
#정치데이터 불러오기
politics_data = pd.read_csv('politics2021-08-23-15-30-22-경남신문/2021-08-23-15-30-22-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(politics_data)):
  politics_data['뉴스내용'][i] = re.sub("[\[\]]","",politics_data['뉴스내용'][i])

#문장단위 토크나이징
politics_sentences  = []
for i in range(0,len(politics_data)):
  politics_sentences.append(kss.split_sentences(politics_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
politics_sentences = sum(politics_sentences , [])

#정치데이터 라벨링(False)
politics = pd.DataFrame(politics_sentences)
politics['label'] = 0

In [ ]:
#경제데이터 불러오기
economy_data = pd.read_csv('economy2021-08-23-15-33-25-경남신문/2021-08-23-15-33-25-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(economy_data)):
  economy_data['뉴스내용'][i] = re.sub("[\[\]]","",economy_data['뉴스내용'][i])

#문장단위 토크나이징
economy_sentences  = []
for i in range(0,len(economy_data)):
  economy_sentences.append(kss.split_sentences(economy_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
economy_sentences = sum(economy_sentences , [])

#경제데이터 라벨링(False)
economy = pd.DataFrame(economy_sentences)
economy['label'] = 0

In [ ]:
#연예데이터 불러오기
entertainment_data = pd.read_csv('entertainment2021-08-23-15-35-12-경남신문/2021-08-23-15-35-12-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(entertainment_data)):
  entertainment_data['뉴스내용'][i] = re.sub("[\[\]]","",entertainment_data['뉴스내용'][i])

#문장단위 토크나이징
entertainment_sentences  = []
for i in range(0,len(entertainment_data)):
  entertainment_sentences.append(kss.split_sentences(entertainment_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
entertainment_sentences = sum(entertainment_sentences , [])

#연예데이터 라벨링(False)
entertainment = pd.DataFrame(entertainment_sentences)
entertainment['label'] = 0

In [ ]:
#스포츠 데이터 불러오기
sports_data = pd.read_csv('sports2021-08-23-15-36-55-경남신문/2021-08-23-15-36-55-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(sports_data)):
  sports_data['뉴스내용'][i] = re.sub("[\[\]]","",sports_data['뉴스내용'][i])

#문장단위 토크나이징
sports_sentences  = []
for i in range(0,len(sports_data)):
  sports_sentences.append(kss.split_sentences(sports_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
sports_sentences = sum(sports_sentences , [])

#스포츠데이터 라벨링(False)
sports = pd.DataFrame(sports_sentences)
sports['label'] = 0

In [ ]:
#문화 데이터 불러오기
culture_data = pd.read_csv('culture2021-08-23-15-38-42-경남신문/2021-08-23-15-38-42-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(culture_data)):
  culture_data['뉴스내용'][i] = re.sub("[\[\]]","",culture_data['뉴스내용'][i])

#문장단위 토크나이징
culture_sentences  = []
for i in range(0,len(culture_data)):
  culture_sentences.append(kss.split_sentences(culture_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
culture_sentences = sum(culture_sentences , [])

#문화데이터 라벨링(False)
culture = pd.DataFrame(culture_sentences)
culture['label'] = 0

In [ ]:
#기술 데이터 불러오기
technology_data = pd.read_csv('technology2021-08-23-15-40-35-경남신문/2021-08-23-15-40-35-경남신문.csv',encoding="utf-8")

#뉴스 내용 시작과 끝에 있는 대괄호 제거
for i in range(0,len(technology_data)):
  technology_data['뉴스내용'][i] = re.sub("[\[\]]","",technology_data['뉴스내용'][i])

#문장단위 토크나이징
technology_sentences  = []
for i in range(0,len(technology_data)):
  technology_sentences.append(kss.split_sentences(technology_data['뉴스내용'][i]))

#리스트 내 리스트 합치기
technology_sentences = sum(technology_sentences , [])

#기술데이터 라벨링(False)
technology = pd.DataFrame(technology_sentences)
technology['label'] = 0

In [ ]:
#정치,경제,연예,스포츠,문화,기술 기사 병합해서 CSV로 저장
notchian = pd.concat([politics,economy,entertainment,sports,culture,technology])
notchian = notchian.rename(columns={0:'문장'})
notchian.to_csv('일반뉴스라벨링.csv')

In [ ]:
#치안, 일반 뉴스데이터 병합
data = pd.concat([chian,notchian])
data.to_csv('뉴스데이터셋.csv')

In [ ]:
news = pd.read_csv('뉴스데이터셋.csv')
news.drop('Unnamed: 0',axis=1,inplace=True)
news

문장  label
0     사천·남해·하동 등 경남 서부해역 해상 치안을 전담할 사천해양경찰서가 내년에 신설될...      1
1     예산안은 임시청사 확보와 주변 연결도로 등 시설 정비, 그리고 준비요원 인건비 등에...      1
2     해경은 예산안이 올해 정기국회를 통과하면 내년 상반기 사천해경 출범을 목표로 하고 있다.      1
3     준비단장인 총경 1명을 비롯해 실무 요원이 부지 확보 등 사전 준비 작업을 추진할 ...      1
4     통영해경은 경남 서부해역에서 남부해역에 이르는 사천 남해 하동 고성 통영 거제 등 ...      1
...                                                 ...    ...
1788           함양의 산삼은 국내뿐만 아니라 국제적으로 그 품질의 우수성이 입증되었다.      0
1789  함양의 우수한 생육 환경에 대한 데이터베이스를 구축하여 인공지능 스마트팜에서 표준화...      0
1790  지능형 기계 산업의 선진국형 정밀기계 제어와 ICT 기술 등은 천연물의 의약품 원료...      0
1791  함안군이 청년농업인의 영농정착 성공모델을 육성해 농업·농촌 발전 비전을 제시하기 위...      0
1792                                             김명현 기자      0

[1793 rows x 2 columns]

## (2)Train set / Test set으로 나누기

In [ ]:
#특정 비율로 데이터셋을 나눌 때, 클래스가 '0'인 데이터와 '1'인 데이터가 골고루 들어가기 위해선 데이터를 랜덤으로 섞어주고 나눠야 한다. 따라서 다음과 같이 코드를 입력해준다.
news_shuffled = news.sample(frac=1).reset_index(drop=True)

In [ ]:
#train data와 test data의 비율을 설정
#train data & test data 로드 
train = news_shuffled[:1300]
test = news_shuffled[1300:]

In [ ]:
train

문장  label
0                첫째, 같은 장소에서 장기간에 걸쳐 지속적으로 이뤄지는 경우가 많다.      1
1     세부적으로 가족 간 감염이 24명, 직장 내 감염이 25명, 지인 9명, 동선 3명이다.      0
2     통계를 인용하지 않더라도 조선산업 호조 시 남해안 일대 지역경제가 큰 호황을 맞았던...      0
3     본채는 3평 크기의 방1개, 2평 크기의 방 3개, 별채에 방2개 규모로 옛날 가옥...      0
4                                                지광하 기자      0
...                                                 ...    ...
1295  그러나 자치경찰제가 본래 취지를 잘 살려 시행된다면 지역 특색을 살린 보다 나은 자...      1
1296          양계협회 부산경남도지회에 따르면 경남지역의 계란 수급은 정상적인 수준이다.      0
1297  군의회의 원 포인트 임시회를 통한 조례 제정과 추경예산 확보로 지원한 창녕형 긴급재...      1
1298              그러나 그 어느 유권자도 정치인을 봉사하는 자리로 보고 있지 않다.      1
1299  32만여㎡ 부지 사업비 567억 투입연구지역·시민 편의시설 조성진해 서부지역 경제 ...      0

[1300 rows x 2 columns]

## (3) Train set 전처리

In [ ]:
#여기서부터 지금까지 전처리한 dataset이 BERT 모델의 입력데이터가 되기 위한 특수한 전처리가 필요하다.
#(1)[CLS]와 [SEP]
# BERT 분류 모델의 경우 각 문장의 앞마다 [CLS]를 붙여 인식시키고, 문장의 종료는 [SEP]를 붙여 인식시킨다. [CLS]을 인식함으로써 문장의 처음이라 알 수 있게 하고, [SEP]을 인식함으로써 문장의 끝을 알 수 있게 하기 위함이다.
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.문장]

In [ ]:
sentences[:5]

['[CLS] 첫째, 같은 장소에서 장기간에 걸쳐 지속적으로 이뤄지는 경우가 많다. [SEP]',
 '[CLS] 세부적으로 가족 간 감염이 24명, 직장 내 감염이 25명, 지인 9명, 동선 3명이다. [SEP]',
 '[CLS] 통계를 인용하지 않더라도 조선산업 호조 시 남해안 일대 지역경제가 큰 호황을 맞았던 당시를 되돌아본다면 그 영향력은 짐작 가능하다. [SEP]',
 '[CLS] 본채는 3평 크기의 방1개, 2평 크기의 방 3개, 별채에 방2개 규모로 옛날 가옥 형태라 아담하다. [SEP]',
 '[CLS] 지광하 기자 [SEP]']

In [ ]:
#한편, '0'과 '1'의 라벨이 들어있는 컬럼을 'labels'이라는 array에 따로 저장한다.

labels = train['label'].values
#labels >> array([0, 0, 1, ..., 0, 1, 0])

In [ ]:
#(2)서브워드 토크나이저 : WordPiece
#단어를 토큰화 할 때, 단어집합에 없는 단어는 더 쪼개서 '##'을 붙여주는 방식
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 첫째, 같은 장소에서 장기간에 걸쳐 지속적으로 이뤄지는 경우가 많다. [SEP]
['[CLS]', '첫', '##째', ',', '같은', '장', '##소', '##에서', '장', '##기', '##간에', '걸쳐', '지', '##속', '##적으로', '이', '##뤄', '##지는', '경우가', '많다', '.', '[SEP]']


In [ ]:
# 이 이후에, 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행해준다.

MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
#(3)어텐션 마스크
#0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것
#예) 패딩된 데이터 = [40311, 9435, 102, 0, 0]
#>>어텐션 마스크 = [1.0, 1.0, 1.0, 0.0, 0.0]
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
#어텐션마스크가 잘 생성되었는지 확인
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
#(4)Train set을 훈련셋과 검증셋으로 분리하기
#데이터가 입력의 형태를 갖췄다면 훈련셋과 검증셋으로 분리해주도록 한다. 어텐션 마스크도 함께 훈련셋과 검증셋으로 분리하고, 데이터를 모두 파이토치 텐서로 변환시킨다.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)
                                                
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)     
                                                       
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)                                                    

In [ ]:
#마지막으로 배치사이즈를 32로 설정하고, 입력데이터, 어텐션 마스크, 라벨을 하나의 데이터로 묶어 train_dataloader, validation_dataloader라는 입력데이터를 생성해준다.
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## (4) Test set 전처리

In [ ]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.문장]

# 라벨 데이터
labels = test['label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## (5) BERT 모델 불러오기 

In [ ]:
#이제 pretrain된 BERT 모델을 불러오도록 하겠다.

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 한편, 하이퍼 파라미터를 설정해주어야 하는데, 파라미터 값은 참조한 코드에 있는 값 그대로 설정했다. 시연용에서의 에폭 수는 간단하게 4로 설정해주었다.
# 실제 프로젝트에서는 epochs = 100으로 진행하였다.

# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

## (6) 모델학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.64
  Training epcoh took: 0:24:00

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:50

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.45
  Training epcoh took: 0:23:58

Running Validation...
  Accuracy: 0.71
  Validation took: 0:00:49

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.31
  Training epcoh took: 0:23:56

Running Validation...
  Accuracy: 0.73
  Validation took: 0:00:50

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:23:57

Running Validation...
  Accuracy: 0.74
  Validation took: 0:00:50

Training complete!


## (7) 테스트셋 평가

In [ ]:
#지금까지 pretrain된 BERT 모델에 추가로 9000개의 한글 대화 데이터를 추가로 학습시켰다. 이제 테스트 셋 데이터로 학습시킨 모델의 정확도를 측정해보도록 하겠다.
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.86
Test took: 0:03:09


## (8) 새로운 문장 테스트

In [ ]:
# 입력하는 문장도 BERT의 입력 형태로 만드는 작업이 필요하기 때문에 우선 아래 코드를 실행해준다.

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks
    
    # 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
# 이제 새로운 문장을 입력할 준비가 끝났다. 먼저 기존 데이터셋에 있는 일상대화 문장을 입력해 보도록 하겠다.

logits = test_sentences(["""(서울=연합뉴스) 김승욱 기자 = 경찰이 진화하는 사이버 범죄에 대응하고자 관련 수사를 강화한다.

20일 연합뉴스 취재에 따르면 경찰청은 전국 18개 시도경찰청 중 11곳에만 있는 과 단위의 사이버수사 조직을 18곳 모두에 설치하는 방안을 검토하고 있다.

아울러 사이버수사과 산하에 사이버미제수사팀을 신설하는 방안도 추진 중이다.


경찰 관계자는 "사이버 범죄가 갈수록 고도화하고 피해도 늘어 수사를 강화하려는 것"이라며 "신설되는 조직에 투입할 인력 규모 등을 협의 중"이라고 말했다.


시대 변화로 생활 공간이 온라인으로 이동하면서 사이버 범죄도 급증하는 추세다.

경찰청은 올해 2월부터 6월까지 5개월간 사기 범죄를 단속해 2만9천881명을 검거했는데, 범죄 유형별로는 사이버 사기가 1만2천84명으로 가장 많았다.

사이버 공간에서는 'n번방'이나 '박사방' 사건처럼 미성년자를 상대로 한 성범죄도 적지 않아 단속 강화 필요성이 꾸준히 제기됐다."""])
print(logits)

if np.argmax(logits) == 1 :
    print("치안 뉴스")
elif np.argmax(logits) == 0 :
    print("일반 뉴스")

[[-2.3707552  1.8792286]]
치안 뉴스


In [ ]:
# 이제 새로운 문장을 입력할 준비가 끝났다. 먼저 기존 데이터셋에 있는 일상대화 문장을 입력해 보도록 하겠다.

logits = test_sentences(["""서울시가 TBS '김어준의 뉴스공장'을 상대로 낸 정정보도 청구에 대해 언론중재위원회가 정정 및 반론보도문 게재를 직권 결정했다.

23일 서울시에 따르면, 언론중재위는 지난 20일 TBS에 서울시 역학조사 관련 보도에 대한 직권조정 결정문을 송달했다. 언론중재위는 '서울시 역학조사 태스크포스(TF) 해체' 관련 보도에 대해선 정정보도를, '역학조사관 인원 축소' 관련 보도에 대해선 반론 보도를 하도록 결정했다.

TBS '김어준의 뉴스공장' 진행자 김어준씨는 지난달 9일과 13일 두 차례에 걸쳐 "서울시가 3~5월보다 역학조사관을 줄였다" "서울시 전담 역학조사 TF를 최근 해체했는데 오 시장 취임 이후인 6월 24일 해체했다"는 발언을 했다.

서울시는 방송 이후 "명백한 가짜뉴스"라고 반박하며 지난달 14일 정정보도를 청구했다. 당시 서울시는 "역학조사관은 한시적으로 채용한 조사관의 퇴사 및 복무만료 등으로 3월 90명에서 4월 73명으로 감소한 뒤, 7월 75명으로 비슷한 수준으로 운영되고 있다"고 설명했다. 서울시는 또 "'역학조사TF'라는 조직을 운영한 적이 없다"며 "지난해 7월 30일 역학조사 전담조직인 '역학조사실'을 신설했고, 현재도 운영 중"이라고 말했다.



TBS는 언론중재위 직권조정 결정을 송달받은 20일로부터 7일 이내에 이의신청을 할 수 있다. 만약 이의신청을 하지 않아 언론중재위 결정이 확정되면 TBS는 확정일로부터 7일 이내에 정정 및 반론보도문을 방송 및 홈페이지에 게재해야 한다."""])
print(logits)

if np.argmax(logits) == 1 :
    print("치안 뉴스")
elif np.argmax(logits) == 0 :
    print("일반 뉴스")

[[ 1.1893443 -1.5322075]]
일반 뉴스
